### How to run the scripts on your computer

First of all, you need to install the libraries:
    
    conda env create -f environment.yml
    
Before running a script in the terminal you need to activate the environment:
    source activate bioinfo-env
    
Then, just do:
    python <SCRIPT.py>

### How to import napoli.py from everywhere...

I would say that the best way to import my scripts from everywhere would be by adding the project directory into the python ENV. 

Obs: by everywhere, I mean: openning a terminal at any directory you want and be able to import the library without errors. Usually, I just go to the main directory and open a terminal there and it works. I didn't worried about creating a script to set the ENV yet. 

Another simple way to do it is:
    
    import os.path
    import sys
    sys.path.append(os.path.dirname(os.getcwd()))
    
    import napoli

#### Let's to code now...

### Discovering which ligands to use

In [3]:
from mol.entry import get_pli_entries
from MyBio.util import download_pdb
from MyBio.PDB.PDBParser import PDBParser

pdb_id = '3QQK'
output_path = "."
download_pdb(pdb_id=pdb_id, output_path=output_path)

pdb_file = '%s/%s.pdb' % (output_path, pdb_id)

# BioPython part: parse a pdb file into a structure object.
PDB_PARSER = PDBParser(PERMISSIVE=True, QUIET=True, FIX_ATOM_NAME_CONFLICT=False, FIX_OBABEL_FLAGS=False)
structure = PDB_PARSER.get_structure(pdb_id, pdb_file)

# The function get_pli_entries() allows you to recover ligands from a PDB file and return a list of strings. 
# So, you'll be able to choose which ligands you want to analyze and then create the Entry objects as showed in 
# the last shell
for entry in get_pli_entries(structure):
    print(entry)

3QQK:A:EDO:490 
3QQK:A:EDO:491 
3QQK:A:EDO:492 
3QQK:A:EDO:493 
3QQK:A:EDO:496 
3QQK:A:X02:497 


### Defining your own Protein-Ligand entries

In [4]:
# A ligand entry is always defined by: pdb_id, chain_id, comp_name, comp_num. 
# There is also an optional parameter comp_icode for setting Icode.

#
# IMPORTANT: the pdb_id property must coincide with the PDB filename.
#
from mol.entry import *
        
# Manually: 
entry = LigandEntry("3QQK", "A", "X02", 497)
# or...
entry = LigandEntry.from_string("3QQK:A:X02:497", sep=":")
     
# Use PDB + molecule files (Mol, Mol2, etc):
# This option allows you to define a separate ligand file as input. 
# This is useful for keeping the ligand charge information, bond order, valence, etc.
entry = MolEntry("my_pdb_id", "Mol-id", "/my/dir/file.mol", mol_obj_type='rdkit')

# If you want to calculate interaction in a whole protein chain:
entry = ChainEntry.from_string("3QQK:A", sep=":")

# From a file:
input_file = "tmp/input_example"
entries = []
# In this example, each line in the input file has an entry.
with open(input_file, "r") as IN:
    for l in IN.readlines():
        entries.append(LigandEntry.from_string(l.strip()))
print(entries)

[<LigandEntry: 3QL8:A:X01:300>, <LigandEntry: 4Y8D:A:49J:401>, <LigandEntry: 5ORV:A:A65:406>]


### Set up the parameters to calculate interactions

In [5]:
import napoli

opt = {}

# The list of entries you defined previously.
opt["entries"] = entries

# Where do you want to save your project.
opt["working_path"] = "tmp/example_project"

# In case the working path you defined above already exists, it allows the script to overwrite the directory.
# Without it, the program will raise an exception. Just be aware that it can remove files from a previous project.
opt["overwrite_path"] = True

# Set where your PDB files are located if they are already available. If you don't have any PDBs, the tool
# will try to download them from the PDB site (https://rcsb.org).
opt["pdb_path"] = "tmp"

# Usually PDB files are download by the program with the format "PDB-ID.pdb". E.g.: 3QQK.pdb.
# However, you can name it the way you prefer, like "3qqk-docking-NAD.pdb". For the latter case, you need
# to set the following flag. It will loose the PDB validation that the program applies on the entries.
#
# OBS: remember that the pdb_id in the entries must coincide with the PDB filename. So, in the example I gave you 
# the corresponding entry should be something like: "3qqk-docking-NAD:A:NAD:1"
#
opt["has_local_files"] = False

# This option is useful when you start working with MolEntry objects and have only one Mol file to all the ligands. 
# So, this option allows the program to load all the molecules at once instead of performing an I/O operation
# for each ligand. This will save you a lot of processing time.
opt["preload_mol_files"] = False

# Define if you need to add Hydrogens or not.
opt["add_hydrogen"] = True
# Controls the pH and how the hydrogens are going to be added. You do not need to define this line.
opt["ph"] = 7.4

# This option is useful when you are working with PDB files. As I said, PDB files does not have charge,
# bond order, nor valence information. So, basically I implemented a sanitization function to correct charges
# and valences for some atoms.
opt["amend_mol"] = True

# Define if you want to work with RDKit or OpenBabel objects. The default is RDKit, so you do not need to 
# change it and, so, you can remove this line. I just put it here for you to know that it exists.
opt["mol_obj_type"] = 'rdkit'

# This flag defines if you want to compute Molecular fingerprints or not.
opt["calc_mfp"] = False

# This flag defines if you want to compute Interaction fingerprints (IFP) or not.
# If your goal is to test different fingerprint parameters, you can set it off.
opt["calc_ifp"] = False

# IFP parameter: number of levels
opt["ifp_num_levels"] = 5
# IFP parameter: radius growth
opt["ifp_radius_step"] = 1
# IFP parameter: size of the fingerprint.
opt["ifp_length"] = 2048

# Here, you need to define the type of entries you are submitting to: protein-ligand (LigandEntry, MolEntry) 
# or protein-protein (ChainEntry) interactions. It controls the type of interactions should be calculated, i.e.,
# the filters the program should apply. For example, in a protein-ligand context I would not be interested
# to see chain-chain interactions. You can use the PLI_PROJECT option by default.
#
# I will change the way I do it in the next days. So, at some point this option will be deprecated. I'll let you
# know when it changes.
#
# Protein-ligand interactions
project_type = napoli.PLI_PROJECT
# Protein-protein interactions
# project_type = napoli.PPI_PROJECT
opt["project_type"] = project_type

# This flag controls if non-covalent interactions should be added. In our cases, we always want it.
opt["add_non_cov"] = True
# This flag controls if covalent, van der Waals, clashes should be considered.
opt["add_atom_atom"] = True
# You can define contacts only by considering if two atoms are near from each other. Not so useful right now.
opt["add_proximal"] = False

### Execute the project defined previously.

In [6]:
from napoli import Local_Project, Fingerprint_Project

# LocalProject: useful if you want to save results locally, and also it is useful if your goal is to interact
# with the interactions and other results after the processing.
#
# In a first moment, you can use this option, because it will be faster for you just to generate different
# fingerprints for pre-computed interactions. So, the idea is: run this cell, and then use the interactions as
# I'm going to show in the next examples.
proj_obj = Local_Project(**opt)

# Fingerprint_Project: useful if you want to generate a lot of fingerprint data into a file. 
# It uses multiprocessing, so it will be faster than Local_Project but you will not be able to interact with the
# results after the files were created...I need to find a way to solve this probs...
# proj_obj = Fingerprint_Project(**opt)

# Run it...
proj_obj()

print()
print("DONE!!!")


DONE!!!


### How to interact with the results.

In [7]:
# If you need to check which properties you can access inside objects in Python use dir().
# With this built-in function you can analyze all objects and see which functions and properties they have.
# print(dir(napoli))

# To analyze the residues and ligands you can use the property 'neighborhood'.
# This property is a list of tuples, in which each tuple contains the entry information and its neighborhood.
proj_obj.neighborhoods

# Each neighborhood contains information from the atom groups identified for a particular entry. 
# Departing from each atom group you can also see its features, atoms, and interactions.
# Example:
for entry, nb in proj_obj.neighborhoods:
    for atm_grp in nb:
        print(atm_grp, atm_grp.features, atm_grp.atoms)
        
# To analyze the interactions you can use the property 'interactions'.
# This property is a list of tuples, in which each tuple contains the entry information and its interactions.
proj_obj.interactions

# The property proj_obj.interactions returns tuples: entry and its interactions
for entry, inters in proj_obj.interactions:
    # The variable 'inters' contains all the interactions found for an entry. So, you can loop over it.
    for i in inters:
        # The first group that is involved in this interaction.
        print(i.atm_grp1)
        # The second group that is involved in this interaction.
        print(i.atm_grp2)
        # The interaction type:
        print(i.type)
        

# If you decided to compute interaction fingerprints, you can access it with the property 'ifps'.
# As you may guess, it returns a list of tuples: one for each entry.
proj_obj.ifps


<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/C>] [<Feature=Electrophile>, <Feature=Atom>] [<NBAtom: 3QL8/0/A/LYS-33/C>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/CE>] [<Feature=Atom>, <Feature=WeakDonor>] [<NBAtom: 3QL8/0/A/LYS-33/CE>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/CB>] [<Feature=Atom>, <Feature=WeakDonor>, <Feature=Hydrophobic>] [<NBAtom: 3QL8/0/A/LYS-33/CB>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/CA>] [<Feature=Atom>, <Feature=WeakDonor>] [<NBAtom: 3QL8/0/A/LYS-33/CA>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/CD>] [<Feature=Atom>, <Feature=WeakDonor>, <Feature=Hydrophobic>] [<NBAtom: 3QL8/0/A/LYS-33/CD>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/NZ>] [<Feature=PositivelyIonizable>, <Feature=Atom>, <Feature=Donor>] [<NBAtom: 3QL8/0/A/LYS-33/NZ>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/CG>] [<Feature=Atom>, <Feature=WeakDonor>, <Feature=Hydrophobic>] [<NBAtom: 3QL8/0/A/LYS-33/CG>]
<AtomGroup: [<NBAtom: 3QL8/0/A/LYS-33/N>] [<Feature=Atom>, <Feature=Donor>] [<NBAtom: 3QL8/0/A/LYS-33/N>]
<AtomGroup: [

[]

### How to plot interactions as a Pymol session

In [22]:
from mol.interaction.depiction import PymolInteractionViewer

# First, create a PymolInteractionViewer object. This Class contains some parameters to set. In general, I don't
# change them, but you can try different things if you want.
piv = PymolInteractionViewer()

# Then, call new_session() passing a list of tuples containing a PDB file to load into Pymol and the interactions
# you want to show using this PDB file as a reference. You need also to define a name for the output...
pse_file = "tmp/output.pse"
print("Number of interactions to plot: %d" % len(proj_obj.interactions))
#piv.new_session(proj_obj.interactions, pse_file)

# You can also define exactly which complex to plot.
# For example: let's say we want just the first five entries.
interactions = proj_obj.interactions[0:2]
print("Number of interactions to plot: %d" % len(interactions))
piv.new_session(interactions, pse_file)

print("DONE!!!!")

# If you decide to plot multiple different PDBs in the same visualization, you may need to align
# the structures before as the coordinate systems are different. If you decide you want to do it, please tell
# me and I can teach you how to do it.

Number of interactions to plot: 3
Number of interactions to plot: 2
 Applying pse_export_version=1.800 compatibility
DONE!!!!


### How to compute fingerprints with different parameters

In [8]:
from mol.interaction.fp.shell import ShellGenerator

# Example
shell_level = 8
radius_growth = 1
ifp_length = 2048

# In this example I'm selecting the first neighborhood to generate a fingerprint. 
# Since, each tuple contains an entry and its neighborhood, you need to select the position 1 in the tuple to 
# select only the neighborhood.
neighborhood = proj_obj.neighborhoods[0][1]

# Then, you need to create a ShellGenerator object with the number of levels and radius size.
shells = ShellGenerator(shell_level, radius_growth)
# So, you can call create_shells() passing as parameter the neighborhood you just selected.
# This function returns a ShellManager object. With this object you can access all created shells.
sm = shells.create_shells(neighborhood)
# Then, you need to transform the created shells into a fingeprint. You can just do the following:
fp = sm.to_fingerprint()
# OBS: If you don't pass any parameter it will create a fingerprint with the current size: 2^32.
# Thus, it is a good idea to reduce the fingerprint size to a desired size as shown below.
fp = sm.to_fingerprint(fold_to_size=ifp_length)

#
#
# Now, let's create a fingerprint for all the entries.
#
#
fps = []
for entry, nb in proj_obj.neighborhoods:
    shells = ShellGenerator(shell_level, radius_growth)
    sm = shells.create_shells(nb)
    fp = sm.to_fingerprint(fold_to_size=ifp_length)
    fps.append(fp)

    
# With the same basic idea you can create different fingerprints using different parameters.
# For example:

params = [(5, 1, 2048), (8, 1, 2048), (5, 2, 1024)]

for shell_level, radius_growth, ifp_length in params:
    fps = []
    for entry, nb in proj_obj.neighborhoods:
        shells = ShellGenerator(shell_level, radius_growth)
        sm = shells.create_shells(nb)
        fp = sm.to_fingerprint(fold_to_size=ifp_length)
        fps.append(fp)
        
    # Do something with the fingerprint...
   
print("DONE!!!!")

DONE!!!!


### How to plot shells as a Pymol session

In [59]:
# First, import PymolShellViewer
from mol.interaction.fp.shell_viewer import PymolShellViewer


# First, create a PymolInteractionViewer object. This Class contains some parameters to set. In general, I don't
# change them, but you can try different things if you want.
psv = PymolShellViewer()

# # Then, call new_session() passing a list of tuples containing a PDB file to load into Pymol and a shell
# # you want to show using this PDB file as a reference. You need also to define a name for the output...
# #
# # Basic usage:
# #
# # pse_file = "output.pse"
# #piv.new_session(shell_tuples, pse_file)

#
# Plot shells for a specific centroid (atom group)
#
#
# Select any atom group. In this 
trgt_atm_grp = None
for s in sm.unique_shells:
    if s.central_atm_grp.has_target():
        trgt_atm_grp = s.central_atm_grp        
# Get shells by a centroid C; it will search for all the shells with center C at all levels.
# The function returns a dictionary in which the keys are the levels and the values are the shells.
# As you only want the shells, I can call the function values() to get only the shells.
trgt_shells = sm.get_shells_by_center(trgt_atm_grp, unique_shells=True).values()
# You can also control if only unique shells should be returned by using the parameter 'unique_shells'.
trgt_shells = sm.get_shells_by_center(trgt_atm_grp, unique_shells=True).values()
# Create the inputs (tuples) to the PymolInteractionViewer object.
shell_tuples = [(s.central_atm_grp.atoms[0].get_parent_by_level("S").pdb_file, s) for s in trgt_shells]
# Plot the shells with Pymol.
psv.new_session(shell_tuples, "tmp/output1.pse")


#
# Plot all unique shells
#
# OBS: plot all the shells may take more time depending on the size of the dataset.
#
trgt_shells = sm.unique_shells
# Create the inputs (tuples) to the PymolInteractionViewer object.
shell_tuples = [(s.central_atm_grp.atoms[0].get_parent_by_level("S").pdb_file, s) for s in trgt_shells]
# Plot the shells with Pymol.
psv.new_session(shell_tuples, "tmp/output2.pse")


#
# Plot all shells in a specific level.
#
#
# Get all the shells in the level L. The function returns a list.
trgt_shells = sm.get_shells_by_level(4)
# You can also control if only unique shells should be returned by using the parameter 'unique_shells'.
trgt_shells = sm.get_shells_by_level(4, unique_shells=True)
# Create the inputs (tuples) to the PymolInteractionViewer object.
shell_tuples = [(s.central_atm_grp.atoms[0].get_parent_by_level("S").pdb_file, s) for s in trgt_shells]
# Plot the shells with Pymol.
psv.new_session(shell_tuples, "tmp/output3.pse")


#
# Plot all shells with a defined identifier obtained from a Fingerprint.
#
# When a shell is created, its identifier is the fingerprint position where the shell sets a bit on.
# Therefore, you can use a bit position to search for the corresponding shell. However, as multiple shells
# can generate the same identifier, always remember that the function get_shells_by_identifier() returns a list.
#
# Let's first create a new fingerprint...
fp = sm.to_fingerprint()
# Then, we need a identifier to search for shells. I select a random identifier here.
identifier = fp.get_on_bits()[5]
# Get all shells with a specific identifier. The function returns a list.
trgt_shells = sm.get_shells_by_identifier(identifier)
# You can also control if only unique shells should be returned by using the parameter 'unique_shells'.
trgt_shells = sm.get_shells_by_identifier(identifier, unique_shells=True)
# Create the inputs (tuples) to the PymolInteractionViewer object.
shell_tuples = [(s.central_atm_grp.atoms[0].get_parent_by_level("S").pdb_file, s) for s in trgt_shells]
# Plot the shells with Pymol.
psv.new_session(shell_tuples, "tmp/output4.pse")

#
# Plot all shells with a defined identifier obtained from a folded Fingerprint.
#
# Now, imagine you have folded your fingerprint to a defined size. What happens is that your fingerprint has
# changed: the positions in the fingerprint is different from the original one. So, the identifiers will not be 
# the same. To select shells based on the new identifiers, you will need to unfold the fingerprint. Let's, do it.
#
# Let's create a fingerprint with the full size.
original_fp = sm.to_fingerprint()
# Now, let's reduce the fingerprint to a smaller size, i.e., you fold it.
# In this example, I exaggerated a lot just to have many collisions.
shorter_fp = original_fp.fold(32)
# This is the identifier we are interested in the folded fingerprint.
short_identifier = shorter_fp.get_on_bits()[25]
# Let's now recover the original identifiers. Yes, in the plural, because when you apply a folding operation some
# bits will collide. Then, it will recover all identifiers that was hashed into the folded bit.
original_identifiers = shorter_fp.unfolding_map[short_identifier]
# Get all shells with a specific identifier. The function returns a list.
# Also, note that I changed the old version of this line to a Iterator as we may have multiple identifiers.
# So, we need to apply the function get_shells_by_identifier in all identifiers.
recovered_shells = []
for i in original_identifiers:
    recovered_shells.append(sm.get_shells_by_identifier(i))

all_shell_tuples = []
# Loop over each set of recovered shells
for shells in recovered_shells:
    # Create the inputs (tuples) to the PymolInteractionViewer object.
    shell_tuples = [(s.central_atm_grp.atoms[0].get_parent_by_level("S").pdb_file, s) for s in shells]
    # Add the shells to the target set.
    all_shell_tuples.extend(shell_tuples)

# Plot the shells with Pymol.
psv.new_session(all_shell_tuples, "tmp/output5.pse")

print("DONE!!!!")

 Applying pse_export_version=1.800 compatibility
 Applying pse_export_version=1.800 compatibility
 Applying pse_export_version=1.800 compatibility
 Applying pse_export_version=1.800 compatibility
 Applying pse_export_version=1.800 compatibility
DONE!!!!
